# INSTRUÇÃO PRÁTICA - IP-P008
## Revisao e consolidaçao do conteúdo de __Pandas__ avançado.
Dados de estações metereologicas de Salvador dos anos de 2003, 2013 e 2023, diponiveis em: [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). 

### Exercicio 1: Tratando series temporais por tempos
1. Carregando dados utilizando `DatetimeIndex`

2. Juntando os dois datasets em um só

3. Analisando os dataframes

### Exercicio 2: Utilizando eval() e query()
1. Comparação de desempenho